linear Spectral Unmixing

In [4]:
import scipy.optimize as opt
import scipy.io as sio
import numpy as np
import scipy.optimize 
import matplotlib.pyplot as plt
from scipy.optimize import nnls
Pavia = sio.loadmat('PaviaU_cube.mat')
HSI = Pavia['X'] #Pavia HSI : 300x200x103

# pure spectra (endmembers) matrix: 103x9
ends = sio.loadmat('PaviaU_endmembers.mat') 
endmembers = ends['endmembers']
ground_truth= sio.loadmat('PaviaU_ground_truth.mat')
labels=ground_truth['y']

(A) <br>
(a) Least Squares

In [ ]:
theta = np.zeros((300,200,9))

In [ ]:
def fit(theta,endmembers):
    return np.dot(endmembers, theta.T)

In [ ]:
def objective(theta, endmembers, HSI):
    return np.linalg.norm(HSI - fit(theta,endmembers))

In [ ]:
reconstruction_error_ls=np.zeros((300,200))
abundance_map_ls=np.zeros((300,200,9))
sum_err_ls=0
for i in range(300):
    for j in range(200):
        if labels[i,j] != 0:
            print("i,j",i,j)
            output=opt.minimize(fun=objective,args=(endmembers,HSI[i,j,:]),
                               x0=theta[i,j,:],method='SLSQP')
            reconstruction_error_ls[i,j]=output.fun
            
            abundance_map_ls[i,j]=output.x
            print("pixel: ",i,j)
            print(output)

In [ ]:
sum_ls_err=0
countnonzero=0
# calculate average of all constructions errors per pixel
for i in range(300):
    for j in range(200):
        if reconstruction_error_ls[i,j].any():
            countnonzero+=1
            sum_ls_err+=reconstruction_error_ls[i,j]


In [ ]:
avg_ls=sum_ls_err/countnonzero
avg_ls

(b) Least squares imposing the sum-to-one constraint

In [ ]:
cons = {'type': 'eq', 'fun': lambda theta: np.sum(theta) - 1}

In [ ]:
reconstruction_error_eq_constr=np.zeros((300,200))
abundance_map_eq_constr=np.zeros((300,200,9))
for i in range(300):
    for j in range(200):
        if labels[i,j] != 0:
            output=opt.minimize(fun=objective,args=(endmembers,HSI[i,j,:]),
                               x0=theta[i,j,:],method='SLSQP',constraints=cons)
            reconstruction_error_eq_constr[i,j]=output.fun
            abundance_map_eq_constr[i,j]=output.x
            print("pixel: ",i,j)
            print(output)

In [ ]:
sum_eq_cons_err=0
countnonzero_eqcons=0
# calculate average of all constructions errors per pixel
for i in range(300):
    for j in range(200):
        if reconstruction_error_eq_constr[i,j].any():
            countnonzero_eqcons+=1
            sum_eq_cons_err+=reconstruction_error_eq_constr[i,j]

avg_eq_cons=sum_eq_cons_err/countnonzero_eqcons
avg_eq_cons

(c) Least squares imposing the non-negativity constraint

In [ ]:
bounds=((0, None),)*9

reconstruction_error_ineq_constr=np.zeros((300,200))
abundance_map_ineq_constr=np.zeros((300,200,9))
for i in range(300):
    for j in range(200):
        if labels[i,j] != 0:
            output=opt.minimize(fun=objective,args=(endmembers,HSI[i,j,:]),
                               x0=theta[i,j,:],method='SLSQP',bounds=bounds)
            reconstruction_error_ineq_constr[i,j]=output.fun
            abundance_map_ineq_constr[i,j]=output.x
            print("pixel: ",i,j)
            print(output)

In [ ]:
sum_ineq_cons_err=0
countnonzero_ineqcons=0
# calculate average of all constructions errors per pixel
for i in range(300):
    for j in range(200):
        if reconstruction_error_ineq_constr[i,j].any():
            countnonzero_ineqcons+=1
            sum_ineq_cons_err+=reconstruction_error_ineq_constr[i,j]

avg_ineq_cons=sum_ineq_cons_err/countnonzero_ineqcons
avg_ineq_cons

(c) Least squares imposing the non-negativity and sum-to-one constraint

In [ ]:
bounds=((0, None),)*9
cons = {'type': 'eq', 'fun': lambda theta: np.sum(theta) - 1}

reconstruction_error_all_constr=np.zeros((300,200))
abundance_map_all_constr=np.zeros((300,200,9))
for i in range(300):
    for j in range(200):
        if labels[i,j] != 0:
            output=opt.minimize(fun=objective,args=(endmembers,HSI[i,j,:]),
                               x0=theta[i,j,:],method='SLSQP',bounds=bounds,constraints=cons)
            reconstruction_error_all_constr[i,j]=output.fun
            abundance_map_all_constr[i,j]=output.x
            print("pixel: ",i,j)
            print(output)

In [ ]:
sum_all_cons_err=0
countnonzero_allcons=0
# calculate average of all constructions errors per pixel
for i in range(300):
    for j in range(200):
        if reconstruction_error_all_constr[i,j].any():
            countnonzero_allcons+=1
            sum_all_cons_err+=reconstruction_error_all_constr[i,j]

avg_all_cons=sum_all_cons_err/countnonzero_allcons
avg_all_cons

(e) Lasso

In [1]:
from sklearn.linear_model import Lasso
lasso = Lasso(alpha=0.2, normalize=True)

* Lasso regression performs L1 regularization, i.e. it adds a factor of sum of absolute value of coefficients in the optimization objective

In [ ]:
reconstruction_error_lasso=np.zeros((300,200))
abundance_map_lasso=np.zeros((300,200,9))

for i in range(300):
    for j in range(200):
        if labels[i,j] != 0:
            lasso.fit(endmembers,HSI[i,j,:])
            err=objective(lasso.coef_, endmembers, HSI[i,j,:])
            reconstruction_error_lasso[i,j]=err
            abundance_map_lasso[i,j]=lasso.coef_
            print("pixel: ",i,j)
            print(reconstruction_error_lasso[i,j])

In [ ]:
sum_all_lasso=0
countnonzero_lasso=0
# calculate average of all constructions errors per pixel
for i in range(300):
    for j in range(200):
        if reconstruction_error_lasso[i,j].any():
            countnonzero_lasso+=1
            sum_all_lasso+=reconstruction_error_lasso[i,j]

avg_lasso=sum_all_lasso/countnonzero_lasso
avg_lasso

(B) Compare the results for the 5 methods applied. We have the following the results:

* Least Squares <br>
  __reconstruction error:__ 335.59216004398417 <br>
  __indicative abundance maps__ <br>
  __pixel(0,14)__array([ 2.40319839e+00, -1.98003744e-01,  1.80754118e+00,  6.54947180e-02,2.62016462e-04, -5.48216886e-02,  3.47221821e-01, -2.76945048e+00,-4.82494537e-01]) <br>
  __pixel(100,180)__ array([-3.50675135, -0.18797048,  1.41721506,  0.04490052,  0.05868581,1.85615715,  1.51750446, -1.10907374,  1.51232401])

* Least squares imposing the sum-to-one constraint <br>
  __reconstruction error:__ 385.598558731914
  __indicative abundance maps__ <br>
  __pixel(0,14)__ array([ 3.17669041e+00, -2.62894368e-01,  2.05246859e+00,  7.52884689e-02,3.02680536e-03, -2.42458337e-02,  1.58568234e-01, -3.37202582e+00, -8.06876489e-01]) <br>
  __pixel(100,180)__ array([ 0.41368273, -0.51668479,  2.65833194,  0.09457088,  0.07271173,2.01084241,  0.56098352, -4.16259194, -0.13184648])

* Least squares imposing the non-negativity constraint <br>
  __reconstruction error:__ 651.3103662374119 <br>
  __indicative abundance maps__ <br>
  __pixel(0,14)__ array([4.11051289e-07, 4.59893931e-08, 1.13491338e-01, 0.00000000e+00,8.69479316e-03, 0.00000000e+00, 7.32247327e-01, 4.44852340e-07,5.59459128e-01]) <br>
  __pixel(100,180)__ array([1.08511163e-07, 1.69296254e-01, 2.03360088e-07, 3.89799958e-01,0.00000000e+00, 3.54908623e-01, 8.91299704e-08, 1.91703161e-07,0.00000000e+00])

* Least squares imposing the non-negativity and sum-to-one constraint <br>
  __reconstruction error:__ 1229.7602033939133 <br>
  __indicative abundance maps__ <br>
  __pixel(0,14)__ array([3.20873861e-20, 0.00000000e+00, 6.42217231e-01, 0.00000000e+00,7.49558697e-02, 2.22960495e-08, 0.00000000e+00, 4.78371015e-10,2.82826913e-01]) <br>
  __pixel(100,180)__ array([8.27034964e-09, 2.04394137e-01, 0.00000000e+00, 3.87863466e-01,1.70488676e-08, 3.13757049e-01, 7.94898794e-09, 0.00000000e+00,9.39853729e-02])

* Lasso <br>
  __reconstruction error__ : 5522.237086140446 <br>
  __indicative abundance maps__ <br>
  pixel(0,14) array([ 0.        ,  0.        ,  0.18552524,  0.01152808, -0.01473208,0.        , -0.        ,  0.        ,  0.09995861])
        
  pixel(100,180) array([-0.        ,  0.        , -0.        ,  0.23101496,  0.02682174,0.84286301, -0.        , -0.        ,  0.74299624])

We can conclude that the reconstruction of the lasso method, which optimize the function by using regularization with L1 norm, is the greatest among the rest of the methods.
Also we can see that when imposing more costraints on the method the reconstruction error gets greater.